# Sky Matching 

<div class="alert-danger">Note: The notebook in this repository 'Initialization.ipynb' goes over many of the basic concepts such as the setup of the environment/package installation and should be read first if you are new to HST images, DrizzlePac, or Astroquery.</div>

## Introduction

AstroDrizzle has the ability to compute and then either subtract or equalize the background in input images in order to match the sky between images. The function allows users to select the algorithm for the sky subtraction using the `skymethod` parameter. 

There are four methods available in sky matching: 'localmin', 'match', 'globalmin', 'globalmin+match'.

By applying `drizzlepac.sky.sky()`, AstroDrizzle will update the keyword `MDRIZSKY` in the headers of the input files. 

The **recommended** use for images with sparse fields that have few astronomical sources is `skymethod = localmin` and for images with complicated backgrounds, such as nebulae and large host galaxies, `skymethod = match` is recommended.

For more information on the specifics of this function please refer to the documentation [here](https://drizzlepac.readthedocs.io/en/deployment/sky.html#drizzlepac.sky.sky).

Below, each of the four methods will be demonstrated using a single example dataset, and differences between the methods will be highlighted. 

In [ ]:
# All imports needed through out this notebook are included at the beginning. 
import glob
import os
import shutil
from shutil import copyfile

from astropy.io import fits
from astroquery.mast import Observations
import bokeh.plotting as bpl
import bokeh.models as bmo
import pandas as pd

from ccdproc import ImageFileCollection

import drizzlepac
from drizzlepac import tweakreg
from drizzlepac import tweakback
from stwcs import updatewcs 

import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from IPython.display import Image

bpl.output_notebook()

## 1. Retrieve Observations using Astroquery

WFC3/IR observations of the Horsehead Nebula obtained in HST proposal 12812 (PI: Levay) will be used for this demonstration. The images have been processed by the HST WFC3 pipeline (calwf3), which includes bias subtraction, dark current correction, cosmic-ray rejection, and flatfielding.

The alignment specifics for this example are based on the example found [here](https://archive.stsci.edu/prepds/heritage/horsehead/readme_HLSP_v3.txt). We will provide explanation about the process as we go. 

The code used to retrieve the data from astroquery is below.

In [ ]:
# Retrieve the observation information.
obs_table = Observations.query_criteria(proposal_id='12812',filters=["F160W"],project='HST')
obs_table.show_in_notebook()

# Figure out the downloadable products:
dataProductsByObservation = Observations.get_product_list(obs_table)
dataProductsByObservation.show_in_notebook()

# Download the flc.fits files: 
obsids = obs_table['obsid']

# Download the data - uncomment to download: 
Observations.download_products(obsids,mrp_only=False,
                               productSubGroupDescription=['FLT','DRZ'])

### Move files to the local directory 

In [ ]:
flt_files = glob.glob('mastDownload/*/*/*flt.fits')

for file in flt_files:
    im = fits.open(file)
    rootname = im[0].header['ROOTNAME']
    im.close()
    dst = rootname + "_flt.fits"
    copyfile(file, dst)

drz_files = glob.glob('mastDownload/*/*/*drz.fits')

for file in drz_files:
    im = fits.open(file)
    rootname = im[0].header['ROOTNAME']
    im.close()
    dst = rootname + "_drz.fits"
    copyfile(file, dst)

### Inspect the image headers

In [ ]:
collect_flt = ImageFileCollection('./', glob_include="*flt.fits", ext=0,
                                  keywords=["asn_id","detector","filter","exptime","postarg1","postarg2"])
    
flt_table = collect_flt.summary
flt_table['exptime'].format='7.1f'
flt_table['postarg1'].format='7.2f'
flt_table['postarg2'].format='7.2f'
flt_table

In [ ]:
collect_drz = ImageFileCollection('./', glob_include="*drz.fits", ext=0,
                                  keywords=["asn_id","detector","filter","exptime"])
    
drz_table = collect_drz.summary
drz_table['exptime'].format='7.1f'
drz_table

## 2. Align the visit-level drizzled data

Exposures obtained within a single HST visit tend to be aligned very well since they use the same guide stars. Thus, we will use TweakReg to align the DRZ files and then use TweakBack to propagate those solutions back to the FLT image headers prior to combining with AstroDrizzle. Making use of the parameter `expand_refcat`, TweakReg will build up an expanded reference catalog on the sky to be used for alignment. For this dataset, we obtain the best results when we tell TweakReg to align the DRZ files in a specific order, and this is achieved by giving a list of files as input. More details on alignment of HST mosaics can be found in the notebook 'mosaics.ipynb' in this repository.
    
    drz.list
    ibxl54030_drz.fits	<-- Note that tile 54 is the reference and is listed first
    ibxl51030_drz.fits  
    ibxl53030_drz.fits
    ibxl55030_drz.fits
    ibxl57030_drz.fits
    ibxl50030_drz.fits
    ibxl52030_drz.fits
    ibxl56030_drz.fits
    ibxl58030_drz.fits
      ____ ____ ____
     |    |    |    |    <-- The 4 tiles with the most overlap are aligned first
     |    | 55 |    |
     |____|____|____|
     |    |    |    |
     | 57 | 54 | 51 |
     |____|____|____|
     |    |    |    |
     |    | 53 |    |
     |____|____|____|
   
      ____ ____ ____
     |    |    |    |    <-- Then the corner tiles are added 
     | 58 |    | 52 |
     |____|____|____|
     |    |    |    |
     |    |    |    |
     |____|____|____|
     |    |    |    |
     | 56 |    | 50 |
     |____|____|____|

This allows TweakReg to start with the tiles with the most overlap first and then build upon that, expanding the reference catalog as each new tile is aligned. 

In [ ]:
tweakreg.TweakReg('@drz.list', 
                  imagefindcfg={'threshold':10,'conv_width':4.5,'peakmin':1}, 
                  minobj=5, 
                  shiftfile=True, 
                  expand_refcat=True, 
                  enforce_user_order=True, 
                  outshifts='shift4_drc.txt',
                  searchrad=3.0,ylimit=0.5, 
                  updatehdr=True, 
                  interactive=False)

Tweakback is then run on the aligned DRZ files to propogate the updated WCS information back to the FLT files.

In [ ]:
tweakback.tweakback('ibxl50030_drz.fits', verbose=True)
tweakback.tweakback('ibxl51030_drz.fits', verbose=True)
tweakback.tweakback('ibxl52030_drz.fits', verbose=True)
tweakback.tweakback('ibxl53030_drz.fits', verbose=True)
tweakback.tweakback('ibxl54030_drz.fits', verbose=True)
tweakback.tweakback('ibxl55030_drz.fits', verbose=True)
tweakback.tweakback('ibxl56030_drz.fits', verbose=True)
tweakback.tweakback('ibxl57030_drz.fits', verbose=True)
tweakback.tweakback('ibxl58030_drz.fits', verbose=True)

## 3. Compare  `skymethod` options in AstroDrizzle

### `skymethod = 'localmin'`

When using AstroDrizzle to compute the sky in each frame, 'localmin' will compute a common sky for all members of an exposure, as described [here](https://drizzlepac.readthedocs.io/en/deployment/sky.html#drizzlepac.sky.sky). This algorithm is recommended when images are dominated by blank sky instead of extended, diffuse sources.

For a typical use, it will compute sky values for each chip/image extension (marked for sky subtraction in the input parameter) in an input image, and it will subtract the previously found minimum sky value from all chips in that image. This process is repeated for each input image.

In the command below, the aligned FLT frames are sky subtracted and drizzled together. Because the WFC3/IR data products are already cleaned of cosmic rays during calwf3 processing, cosmic-ray rejection is turned off in AstroDrizzle by setting the parameters `driz_separate`, `median`, `blot`, and `driz_cr` to 'False'.

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle('*flt.fits', 
                                     output='f160w_localmin',
                                     preserve=False, 
                                     skymethod='localmin', 
                                     driz_separate=False, 
                                     median=False, 
                                     blot=False, 
                                     driz_cr=False, 
                                     final_bits='64',
                                     final_wcs=True, 
                                     final_rot=257.)

### `skymethod = 'match'`

When `skymethod` is set to ‘match’, differences in sky values between images in common sky regions will be computed. Thus, sky values will be relative (delta) to the sky computed in one of the input images whose sky value will be set to and reported as 0. This setting “equalizes” sky values between the images in large mosaics. 

This is the **recommended** setting for images containing diffuse sources (e.g., galaxies, nebulae) covering significant parts of the image.

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle('*flt.fits', 
                                     output='f160w_match',
                                     preserve=False, 
                                     skymethod='match',
                                     driz_separate=False, 
                                     median=False, 
                                     blot=False, 
                                     driz_cr=False, 
                                     final_bits='64',
                                     final_wcs=True, 
                                     final_rot=257.)

### `skymethod = 'globalmin+match'`

When `skymethod` is set to ‘globalmin+match', AstroDrizzle will first find a minimum “global” sky value in all input images and then use the ‘match’ method to equalize sky values between images.

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle('*flt.fits', 
                                     output='f160w_globalmin_match',
                                     preserve=False, 
                                     skymethod='globalmin+match',
                                     driz_separate=False, 
                                     median=False, 
                                     blot=False, 
                                     driz_cr=False, 
                                     final_bits='64',
                                     final_wcs=True, 
                                     final_rot=257.)

### `skymethod = 'globalmin'`

When `skymethod` is set to ‘globalmin’, a common sky value will be computed for all exposures. AstroDrizzle will compute sky values for each chip/image extension, find the minimum sky value from all the exposures, and then subtract that minimum sky value from all chips in all images. 

This method may be useful when input images already have matched background values.

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle('*flt.fits', 
                                     output='f160w_globalmin',     
                                     preserve=False, skymethod='globalmin',     
                                     driz_separate=False, 
                                     median=False, 
                                     blot=False, 
                                     driz_cr=False, 
                                     final_bits='64',
                                     final_wcs=True, 
                                     final_rot=257.)

## 4. Compare the MDRIZSKY values for each method

The following MDRISKY values were derived for each of the four test cases: 

Rootname |  ASN ID |   Orientat   | MDRIZSKY localmin  |MDRIZSKY - Match|MDRIZSKY - Globalmin|MDRIZSKY - Globalmin_Match
---------|---------|--------------|--------------------|----------------|--------------------|---------------
ibxl50clq|IBXL50030| 170.6670349  |     1.15010715     |   0.02348762   |     0.79616243     |    0.79161596
ibxl50cqq|IBXL50030| 170.66703485 |     1.16239488     |   0.01991268   |     0.79616243     |    0.78804102
ibxl51eoq|IBXL51030| 166.65263374 |     1.17368317     |   0.03672888   |     0.79616243     |    0.80485721 
ibxl51etq|IBXL51030| 166.65263371 |     1.17073488     |   0.02612076   |     0.79616243     |    0.79424910
ibxl52k0q|IBXL52030| 166.66514055 |     0.80133963     |   0.03321128   |     0.79616243     |    0.80133962
ibxl52k5q|IBXL52030| 166.66514048 |     0.79616243     |   0.02150525   |     0.79616243     |    0.78963359
ibxl53kxq|IBXL53030| 165.60829152 |     1.20236218     |   0.04025653   |     0.79616243     |    0.80838487
ibxl53l9q|IBXL53030| 165.60829155 |     1.1834859      |   0.01995461   |     0.79616243     |    0.78808295
ibxl54bgq|IBXL54030| 166.63124069 |     1.16979861     |   0.02172938   |     0.79616243     |    0.78985772
ibxl54blq|IBXL54030| 166.63124069 |     1.1632545      |   0.01651723   |     0.79616243     |    0.78464557
ibxl55f0q|IBXL55030| 166.62229657 |     1.00639629     |   0.01668276   |     0.79616243     |    0.78481110
ibxl55f5q|IBXL55030| 166.62229656 |     1.00930452     |   0.01690617   |     0.79616243     |    0.78503450 
ibxl56huq|IBXL56030| 162.58958453 |     1.3632288      |   0.02503881   |     0.79616243     |    0.79316715
ibxl56i2q|IBXL56030| 162.58958458 |     1.3619746      |   0.02014752   |     0.79616243     |    0.78827586
ibxl57adq|IBXL57030| 166.60915246 |     1.13295722     |   0.00687557   |     0.79616243     |    0.77500390
ibxl57aiq|IBXL57030| 166.60915247 |     1.11978412     |   0.00589288   |     0.79616243     |    0.77402122 
ibxl58sqq|IBXL58030| 166.61403565 |     0.80498779     |   0.00166623   |     0.79616243     |    0.76979457
ibxl58svq|IBXL58030| 166.61403566 |     0.79938716     |   0.0          |     0.79616243     |    0.76812834


These computed sky values can be visualized in the plot below. To reiterate, the MDRIZSKY keyword reports the value subtracted from each image, and not the sky level itself. Thus the values for `skymethod='match'` are close to zero. We also note that varying background levels across the individual tiles result in inaccurate sky background determination when `skymethod='match'` and thus a mismatched sky in the final mosaic.

<img src="MDRIZSKY_Values.png" width="600">

Below we provide a gif comparing the upper portion of the final drizzled image. We cycle through three of the versions that use different skymethod algorithms:  

![SegmentLocal](labeled_local_globalmatch_match.gif)

Below is the code used to compare results from the various skymatching methods. This creates an output csv file that allows the user create a plot like the one above to compare the `MDRIZSKY` values. 

In [ ]:
im = fits.open("f160w_globalmin_drz_sci.fits")
rootname = im[1].data['ROOTNAME']
asn_id = im[1].data['ASN_ID']
orientat = im[1].data['ORIENTAT']
mdrizsky = im[1].data['MDRIZSKY']
im.close()

im = fits.open("f160w_globalmin_match_drz_sci.fits")
globalmatchrootname = im[1].data['ROOTNAME']
asn_id = im[1].data['ASN_ID']
orientat = im[1].data['ORIENTAT']
globalmatchmdrizsky = im[1].data['MDRIZSKY']
im.close()

im = fits.open("f160w_match_drz_sci.fits")
matchrootname = im[1].data['ROOTNAME']
asn_id = im[1].data['ASN_ID']
orientat = im[1].data['ORIENTAT']
matchmdrizsky = im[1].data['MDRIZSKY']
im.close()

im = fits.open("f160w_localmin_drz_sci.fits")
localrootname = im[1].data['ROOTNAME']
asn_id = im[1].data['ASN_ID']
orientat = im[1].data['ORIENTAT']
localmdrizsky = im[1].data['MDRIZSKY']
im.close()

mkdrisky_val = pd.DataFrame(
    {'Rootname': rootname,
     'globalmin': mdrizsky,
     'globalmin_match': globalmatchmdrizsky,
     'match': matchmdrizsky,
     'local': localmdrizsky})

mkdrisky_val['norm_local'] = mkdrisky_val['local']
mkdrisky_val['norm_match'] = mkdrisky_val['match']
mkdrisky_val['norm_global'] = mkdrisky_val['globalmin']
mkdrisky_val['norm_globalminmatch'] = mkdrisky_val['globalmin_match']

mkdrisky_val.to_csv("example_mkdrisky_results.csv")

In [ ]:
mkdrisky_val = pd.read_csv("example_mkdrisky_results.csv")
source = bpl.ColumnDataSource(mkdrisky_val)

# create figure and plot
p = bpl.figure(title="MDRIZSKY Values of Different Sky Matching Techniques", y_range=(-0.1, 1.25))
p.xaxis.axis_label = "Individual Images"
p.yaxis.axis_label = "MDRIZSKY Value"
p.scatter(x='index', y='globalmin_match', legend="Globalmin + Match", size=7, color="magenta", source=source)
p.scatter(x='index', y='match', legend="Match", size=7, color="navy", source=source)
p.scatter(x='index', y='local', legend="Localmin", size=7, color="olive", source=source)
p.scatter(x='index', y='globalmin', legend="Globalmin", size=7,  color="orange", source=source)

p.legend.location = "center_left"

bpl.show(p)

## 5. Display the full 'matched' mosaic 

In [ ]:
sci=fits.getdata('f160w_match_drz_sci.fits')
fig=plt.figure(figsize=(14,14))
plt.imshow(sci, vmin=-0.4, vmax=3, cmap='Greys_r', origin='lower')

# About this Notebook

    Author: C. Martlin, STScI WFC3 Team  
    Updated: December, 14 2018